In [76]:
import pandas as pd
from glob import glob
from time import strftime, sleep
import numpy as np
import statsmodels.api as sm
from datetime import datetime
from pandas_datareader import data as pdr
from pandas.tseries.offsets import BDay
import yfinance as yf
yf.pdr_override()

In [77]:
def add_transaction(ticker, quantity, type, price, excel_file_name, date=datetime.now(),  fee=0):
    # Read existing data from Excel file
    try:
        df = pd.read_excel(excel_file_name)
    except FileNotFoundError:
        # If file doesn't exist, create a new DataFrame
        df = pd.DataFrame(columns=['date', 'type' ,'ticker', 'quantity', 'price', 'fees',
                                    'transact_val', 'last_occurence', 'cashflow', 'prev_units',
                                    'cml_units', 'prev_cost', 'cml_cost', 'cost_unit', 
                                    'cost_transact', 'gain_loss', 'yield', 'avg_price'
                                    ])


    # compute value of the transaction
    transact_val = price * quantity

    # gives the index of the previous transaction of a given stock (chatgpt)
    last_occurence = df[df['ticker'] == ticker].index.max() if ticker in df['ticker'].unique() else None

    # gives the cash-flow of the transaction
    if type == 'Buy':
        cashflow = transact_val * -1
    else:
        cashflow = transact_val

    cml_unit = quantity
    
    if last_occurence is not None:
        # gives the quantities of the last transaction of a stock
        prev_unit = df['quantity'].iloc[last_occurence]

        # gives buying price
        prev_cost = prev_unit * df['price'].iloc[last_occurence]

        if type == 'Sell':
            cost_unit = df['price'].iloc[last_occurence]
            cost_transact = quantity * cost_unit
            gain_loss = transact_val - cost_transact
            y = gain_loss / cost_transact
            cml_unit = df['cml_units'].iloc[last_occurence] - quantity
            cml_cost = prev_cost - cost_transact
        else:
            cost_unit = 0
            cost_transact = 0
            gain_loss = 0
            y = 0
            cml_unit = df['cml_units'].iloc[last_occurence] + quantity
            cml_cost = df['cml_cost'].iloc[last_occurence] + transact_val

        avg_price = cml_cost / cml_unit


    # Append new transaction to the DataFrame
        new_transaction = pd.DataFrame({
            'date': [date],
            'type': [type],
            'ticker': [ticker],
            'quantity': [quantity],
            'price': [price],
            'fees': [fee],
            'transact_val': [transact_val],
            'last_occurence': [last_occurence],
            'cashflow': [cashflow],
            'prev_units': [prev_unit],
            'cml_units': [cml_unit],
            'prev_cost': [prev_cost],
            'cml_cost': [cml_cost],
            'cost_unit': [cost_unit],
            'cost_transact': [cost_transact],
            'gain_loss': [gain_loss],
            'yield': [y],
            'avg_price': [avg_price]
        })
    else:
        new_transaction = pd.DataFrame({
            'date': [date],
            'type': [type],
            'ticker': [ticker],
            'quantity': [quantity],
            'price': [price],
            'fees': [fee],
            'transact_val': [transact_val],
            'last_occurence': [last_occurence],
            'cashflow': [cashflow],
            'cml_units': [cml_unit],
            'cml_cost': [transact_val]
        })
    df = pd.concat([df, new_transaction], ignore_index=True)

        # Save DataFrame to Excel file
    df.to_excel(excel_file_name, index=False)
    print("Transaction added successfully.")

In [83]:
# Simulation of orders for the exemple
if __name__ == "__main__":
    # add_transaction('AAPL', 20, 'Buy', 179.48, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('GOOGL', 20, 'Buy', 144.99, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('TSLA', 20, 'Buy', 399.92, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('AMZN', 20, 'Buy', 170.40, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('V', 20, 'Buy', 217.63, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('JPM', 20, 'Buy', 150.40, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('JNJ', 20, 'Buy', 161, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('MA', 20, 'Buy', 365.37, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('UNH', 20, 'Buy', 486.65, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('PG', 20, 'Buy', 153.05, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('PYPL', 20, 'Buy', 194.94, 'portfolio_transactions.xlsx', date=datetime(2022, 1, 3, 19, 0, 0))
    # add_transaction('AAPL', 5, 'Sell', 169.94, 'portfolio_transactions.xlsx', date=datetime(2022, 2, 10, 19, 0, 0))
    # add_transaction('AMZN', 10, 'Sell', 156.51, 'portfolio_transactions.xlsx', date=datetime(2022, 3, 10, 19, 0, 0))
    # add_transaction('V', 5, 'Sell', 211.40, 'portfolio_transactions.xlsx', date=datetime(2022, 4, 11, 19, 0, 0))
    # add_transaction('AAPL', 10, 'Buy', 150.35, 'portfolio_transactions.xlsx', date=datetime(2022, 5, 9, 19, 0, 0))
    # add_transaction('V', 5, 'Buy', 203.30, 'portfolio_transactions.xlsx', date=datetime(2022, 6, 9, 19, 0, 0))
    add_transaction('TSLA', 10, 'Sell', 179.24, 'portfolio_transactions.xlsx', date=datetime.now())

Transaction added successfully.


In [84]:
 # simple function to make headers nicer
def clean_header(df): #str is pd function that allows you to play with the column and row labels (not sure about rows)
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace('(', '').str.replace(')', '').str.replace(' ', '_').str.replace('_/_', '/')

# timestamp for file names
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now # getter that returns a datetime object of current day and hour

In [85]:
all_transactions = pd.read_excel('/Users/ahmadroukain/PycharmProjects/pythonProject4/AP/Project/portfolio_transactions.xlsx')
# all_transactions.date = pd.to_datetime(all_transactions.date, format='%d/%m/%Y')

In [86]:
all_tickers = list(all_transactions['ticker'].unique())
# some tickers may have been delisted. need to blacklist them here
blacklist = ['VSLR', 'HTZ', 'FB', 'VIAC', 'TWTR', 'XLNX']
filt_tickers = [tick for tick in all_tickers if tick not in blacklist]
print('You traded {} different stocks'.format(len(all_tickers)))

You traded 11 different stocks


In [87]:
# creates a new file with the filtred stocks
final_filtered = all_transactions[~all_transactions.ticker.isin(blacklist)]
final_filtered

,date,type,ticker,quantity,price,fees,transact_val,last_occurence,cashflow,prev_units,cml_units,prev_cost,cml_cost,cost_unit,cost_transact,gain_loss,yield,avg_price
0,2022-01-03 19:00:00.000,Buy,AAPL,20,179.48,0,3589.6,NaN,-3589.6,NaN,20,NaN,3589.60,NaN,NaN,NaN,NaN,NaN
1,2022-01-03 19:00:00.000,Buy,GOOGL,20,144.99,0,2899.8,NaN,-2899.8,NaN,20,NaN,2899.80,NaN,NaN,NaN,NaN,NaN
2,2022-01-03 19:00:00.000,Buy,TSLA,20,399.92,0,7998.4,NaN,-7998.4,NaN,20,NaN,7998.40,NaN,NaN,NaN,NaN,NaN
3,2022-01-03 19:00:00.000,Buy,AMZN,20,170.40,0,3408.0,NaN,-3408.0,NaN,20,NaN,3408.00,NaN,NaN,NaN,NaN,NaN
4,2022-01-03 19:00:00.000,Buy,V,20,217.63,0,4352.6,NaN,-4352.6,NaN,20,NaN,4352.60,NaN,NaN,NaN,NaN,NaN
5,2022-01-03 19:00:00.000,Buy,JPM,20,150.40,0,3008.0,NaN,-3008.0,NaN,20,NaN,3008.00,NaN,NaN,NaN,NaN,NaN
6,2022-01-03 19:00:00.000,Buy,JNJ,20,161.00,0,3220.0,NaN,-3220.0,NaN,20,NaN,3220.00,NaN,NaN,NaN,NaN,NaN
7,2022-01-03 19:00:00.000,Buy,MA,20,365.37,0,7307.4,NaN,-7307.4,NaN,20,NaN,7307.40,NaN,NaN,NaN,NaN,NaN
8,2022-01-03 19:00:00.000,Buy,UNH,20,486.65,0,9733.0,NaN,-9733.0,NaN,20,NaN,9733.00,NaN,NaN,NaN,NaN,NaN
9,2022-01-03 19:00:00.000,Buy,PG,20,153.05,0,3061.0,NaN,-3061.0,NaN,20,NaN,3061.00,NaN,NaN,NaN,NaN,NaN


In [88]:
ly = datetime.today().year-1
today = datetime.today()
start_sp = datetime(2022, 1, 3, 19, 0, 0)
end_sp = today
start_stocks = datetime(2022, 1, 3, 19, 0, 0)
end_stocks = today
start_ytd = datetime(ly, 12, 31) + BDay(1)

def get(tickers, startdate, enddate):
    def data(ticker):
        return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
    datas = map(data, tickers)
    return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))
               
all_data = get(filt_tickers, start_stocks, end_stocks)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [89]:
clean_header(all_data)

In [90]:
# saving all stock prices individually to the specified folder
for tick in filt_tickers:
    all_data.loc[tick].to_csv('/Users/ahmadroukain/PycharmProjects/pythonProject4/AP/Project{}_price_hist.csv'.format(tick))

In [91]:
MEGA_DICT = {}  # you have to create it first
min_date = '2020-01-01'  # optional
TX_COLUMNS = ['date','ticker', 'cashflow', 'cml_units', 'cml_cost', 'gain_loss']
tx_filt = all_transactions[TX_COLUMNS]  # keeping just the most relevant ones for now

for ticker in filt_tickers:
    prices_df = all_data[all_data.index.get_level_values('ticker').isin([ticker])].reset_index()
    ## Can add more columns like volume!
    PX_COLS = ['date', 'adj_close']
    prices_df = prices_df[prices_df.date >= min_date][PX_COLS].set_index(['date'])
    # Making sure we get sameday transactions
    tx_df = tx_filt[tx_filt.ticker==ticker].groupby('date').agg({'cashflow': 'sum',
                                                                 'cml_units': 'last',
                                                                 'cml_cost': 'last',
                                                                 'gain_loss': 'sum'})
    # Merging price history and transactions dataframe
    tx_and_prices = pd.merge(prices_df, tx_df, how='outer', left_index=True, right_index=True).fillna(0)
    # This is to fill the days that were not in our transaction dataframe
    tx_and_prices['cml_units'] = tx_and_prices['cml_units'].replace(to_replace=0, method='ffill')
    tx_and_prices['cml_cost'] = tx_and_prices['cml_cost'].replace(to_replace=0, method='ffill')
    tx_and_prices['gain_loss'] = tx_and_prices['gain_loss'].replace(to_replace=0, method='ffill')
    # Cumulative sum for the cashflow
    tx_and_prices['cashflow'] = tx_and_prices['cashflow'].cumsum()
    
    tx_and_prices['avg_price'] = (tx_and_prices['cml_cost']/tx_and_prices['cml_units'])
    tx_and_prices['mktvalue'] = (tx_and_prices['cml_units']*tx_and_prices['adj_close'])
    tx_and_prices = tx_and_prices.add_prefix(ticker+'_')
    # Once we're happy with the dataframe, add it to the dictionary
    MEGA_DICT[ticker] = tx_and_prices.round(3)

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3249509493.py:19: FutureWarning:

The 'method' keyword in Series.replace is deprecated and will be removed in a future version.

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3249509493.py:20: FutureWarning:

The 'method' keyword in Series.replace is deprecated and will be removed in a future version.

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3249509493.py:21: FutureWarning:

The 'method' keyword in Series.replace is deprecated and will be removed in a future version.

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3249509493.py:19: FutureWarning:

The 'method' keyword in Series.replace is deprecated and will be removed in a future version.

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3249509493.py:20: FutureWarning:

The 'method' keyword in Series.replace is deprecated and will be removed in a future version.

/var/folders/zq/cl9wyfm13sq150

In [92]:
MEGA_DF = pd.concat(MEGA_DICT.values(), axis=1)
MEGA_DF.to_csv('/Users/ahmadroukain/PycharmProjects/pythonProject4/AP/Project/MEGA_DF_{}.csv'.format(get_now()))  # optional
MEGA_DF.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 608 entries, 2022-01-03 19:00:00 to 2024-05-26 23:02:48.310000
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AAPL_adj_close   604 non-null    float64
 1   AAPL_cashflow    604 non-null    float64
 2   AAPL_cml_units   604 non-null    float64
 3   AAPL_cml_cost    604 non-null    float64
 4   AAPL_gain_loss   604 non-null    float64
 5   AAPL_avg_price   604 non-null    float64
 6   AAPL_mktvalue    604 non-null    float64
 7   GOOGL_adj_close  602 non-null    float64
 8   GOOGL_cashflow   602 non-null    float64
 9   GOOGL_cml_units  602 non-null    float64
 10  GOOGL_cml_cost   602 non-null    float64
 11  GOOGL_gain_loss  602 non-null    float64
 12  GOOGL_avg_price  602 non-null    float64
 13  GOOGL_mktvalue   602 non-null    float64
 14  TSLA_adj_close   603 non-null    float64
 15  TSLA_cashflow    603 non-null    float64
 16  TSLA_cml_units   6

In [93]:
last_file = glob('/Users/ahmadroukain/PycharmProjects/pythonProject4/AP/Project/MEGA*.csv')[-1] # path to file in the folder
print(last_file[-(len(last_file))+(last_file.rfind('/')+1):])
MEGA_DF = pd.read_csv(last_file)
MEGA_DF['date'] = pd.to_datetime(MEGA_DF['date'])
MEGA_DF.set_index('date', inplace=True)

MEGA_DF_2024-05-26_23h05m.csv


In [94]:
portf_allvalues = MEGA_DF.filter(regex='mktvalue').fillna(0) #  getting just the market value of each ticker
portf_allvalues['portf_value'] = portf_allvalues.sum(axis=1) # summing all market values
portf_allvalues['portf_value']

date
2022-01-03 19:00:00.000        0.000
2022-01-04 00:00:00.000    51923.134
2022-01-05 00:00:00.000    51020.650
2022-01-06 00:00:00.000    50421.646
2022-01-07 00:00:00.000    49847.819
                             ...    
2024-05-21 00:00:00.000    50765.450
2024-05-22 00:00:00.000    50438.800
2024-05-23 00:00:00.000    49747.500
2024-05-24 00:00:00.000    49811.400
2024-05-26 23:02:48.310        0.000
Name: portf_value, Length: 608, dtype: float64

In [95]:
MEGA_DF

,AAPL_adj_close,AAPL_cashflow,AAPL_cml_units,AAPL_cml_cost,AAPL_gain_loss,AAPL_avg_price,AAPL_mktvalue,GOOGL_adj_close,GOOGL_cashflow,GOOGL_cml_units,...,PG_gain_loss,PG_avg_price,PG_mktvalue,PYPL_adj_close,PYPL_cashflow,PYPL_cml_units,PYPL_cml_cost,PYPL_gain_loss,PYPL_avg_price,PYPL_mktvalue
date,,,,,,,,,,,,,,,,,,,,,
2022-01-03 19:00:00.000,0.000,-3589.6,20.0,3589.6,0.0,179.480,0.000,0.000,-2899.8,20.0,...,0.0,153.05,0.000,0.00,-3898.8,20.0,3898.8,0.0,194.94,0.0
2022-01-04 00:00:00.000,177.203,-3589.6,20.0,3589.6,0.0,179.480,3544.064,144.400,-2899.8,20.0,...,0.0,153.05,3071.859,191.14,-3898.8,20.0,3898.8,0.0,194.94,3822.8
2022-01-05 00:00:00.000,172.490,-3589.6,20.0,3589.6,0.0,179.480,3449.793,137.775,-2899.8,20.0,...,0.0,153.05,3085.764,187.16,-3898.8,20.0,3898.8,0.0,194.94,3743.2
2022-01-06 00:00:00.000,169.610,-3589.6,20.0,3589.6,0.0,179.480,3392.204,137.747,-2899.8,20.0,...,0.0,153.05,3059.832,192.27,-3898.8,20.0,3898.8,0.0,194.94,3845.4
2022-01-07 00:00:00.000,169.778,-3589.6,20.0,3589.6,0.0,179.480,3395.557,137.017,-2899.8,20.0,...,0.0,153.05,3058.141,187.60,-3898.8,20.0,3898.8,0.0,194.94,3752.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-21 00:00:00.000,192.350,-4243.4,25.0,4195.7,-47.7,167.828,4808.750,177.850,-2899.8,20.0,...,0.0,153.05,3367.000,64.10,-3898.8,20.0,3898.8,0.0,194.94,1282.0
2024-05-22 00:00:00.000,190.900,-4243.4,25.0,4195.7,-47.7,167.828,4772.500,176.380,-2899.8,20.0,...,0.0,153.05,3356.200,62.19,-3898.8,20.0,3898.8,0.0,194.94,1243.8
2024-05-23 00:00:00.000,186.880,-4243.4,25.0,4195.7,-47.7,167.828,4672.000,173.550,-2899.8,20.0,...,0.0,153.05,3309.800,61.58,-3898.8,20.0,3898.8,0.0,194.94,1231.6


In [96]:
# For the S&P500 price return
sp500 = pdr.get_data_yahoo('^GSPC', start_stocks, end_sp)
clean_header(sp500)

[*********************100%%**********************]  1 of 1 completed


In [54]:
#getting the pct change
portf_allvalues = portf_allvalues.join(sp500['adj_close'], how='inner')
portf_allvalues.rename(columns={'adj_close': 'sp500_mktvalue'}, inplace=True)
portf_allvalues['ptf_value_pctch'] = (portf_allvalues['portf_value'].pct_change()*100).round(2)
portf_allvalues['sp500_pctch'] = (portf_allvalues['sp500_mktvalue'].pct_change()*100).round(2)
portf_allvalues['ptf_value_diff'] = (portf_allvalues['portf_value'].diff()).round(2)
portf_allvalues['sp500_diff'] = (portf_allvalues['sp500_mktvalue'].diff()).round(2)

In [55]:
# KPI's for portfolio
kpi_portfolio7d_abs = portf_allvalues.tail(7).ptf_value_diff.sum().round(2)
kpi_portfolio15d_abs = portf_allvalues.tail(15).ptf_value_diff.sum().round(2)
kpi_portfolio30d_abs = portf_allvalues.tail(30).ptf_value_diff.sum().round(2)
kpi_portfolio200d_abs = portf_allvalues.tail(200).ptf_value_diff.sum().round(2)
kpi_portfolio7d_pct = (kpi_portfolio7d_abs/portf_allvalues.tail(7).portf_value[0]).round(3)*100
kpi_portfolio15d_pct = (kpi_portfolio15d_abs/portf_allvalues.tail(15).portf_value[0]).round(3)*100
kpi_portfolio30d_pct = (kpi_portfolio30d_abs/portf_allvalues.tail(30).portf_value[0]).round(3)*100
kpi_portfolio200d_pct = (kpi_portfolio200d_abs/portf_allvalues.tail(200).portf_value[0]).round(3)*100

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/2739045084.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/2739045084.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/2739045084.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipy

In [56]:
# KPI's for S&P500
kpi_sp500_7d_abs = portf_allvalues.tail(7).sp500_diff.sum().round(2)
kpi_sp500_15d_abs = portf_allvalues.tail(15).sp500_diff.sum().round(2)
kpi_sp500_30d_abs = portf_allvalues.tail(30).sp500_diff.sum().round(2)
kpi_sp500_200d_abs = portf_allvalues.tail(200).sp500_diff.sum().round(2)
kpi_sp500_7d_pct = (kpi_sp500_7d_abs/portf_allvalues.tail(7).sp500_mktvalue[0]).round(3)*100
kpi_sp500_15d_pct = (kpi_sp500_15d_abs/portf_allvalues.tail(15).sp500_mktvalue[0]).round(3)*100
kpi_sp500_30d_pct = (kpi_sp500_30d_abs/portf_allvalues.tail(30).sp500_mktvalue[0]).round(3)*100
kpi_sp500_200d_pct = (kpi_sp500_200d_abs/portf_allvalues.tail(200).sp500_mktvalue[0]).round(3)*100

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3746021140.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3746021140.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/3746021140.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipy

In [57]:
rf = pdr.get_data_yahoo('^IRX', start_stocks, datetime.now())
rf = rf['Adj Close']
rf

[*********************100%%**********************]  1 of 1 completed


Date
2022-01-04    0.080
2022-01-05    0.085
2022-01-06    0.090
2022-01-07    0.088
2022-01-10    0.090
              ...  
2024-05-20    5.240
2024-05-21    5.240
2024-05-22    5.245
2024-05-23    5.245
2024-05-24    5.245
Name: Adj Close, Length: 601, dtype: float64

In [58]:
# Sort the portfolio value percentage changes in descending order
sorted_ptf_value_diff = portf_allvalues.ptf_value_pctch.sort_values(ascending=False)

# Calculate the 1-day Value at Risk (VaR) at 95%
vaR_95 = np.percentile(sorted_ptf_value_diff.dropna(), 5).round(1)

# Sort the portfolio value percentage changes in descending order again
sorted_ptf_value_pctch = portf_allvalues.ptf_value_pctch.sort_values(ascending=False)

# Drop NaN values and exclude the first element
sorted_ptf_value_pctch = sorted_ptf_value_pctch.dropna().iloc[1:]

# Calculate the index for the 95% expected shortfall (ES)
index = int((1 - 0.95) * len(sorted_ptf_value_pctch))

# Calculate the expected shortfall (ES) at 95%
es_95 = -np.mean(sorted_ptf_value_pctch[:index]).round(1)

# Compute the volatility of the portfolio
volatility = np.std(portf_allvalues.ptf_value_pctch).round(1)

# Calculate the tracking error (TE) relative to the SP500
te = portf_allvalues.ptf_value_pctch.iloc[6:] / 100 - portf_allvalues.sp500_pctch.iloc[6:] / 100
te = np.std(te, ddof=1)

# Calculate cumulative returns of the portfolio
cumulative_returns = (1 + portf_allvalues.ptf_value_pctch.iloc[6:] / 100).cumprod()

# Calculate the running maximum of cumulative returns
running_max = np.maximum.accumulate(cumulative_returns)

# Calculate drawdowns from the running maximum
drawdowns = (cumulative_returns - running_max) / running_max

# Calculate the maximum drawdown
max_drawdown = np.min(drawdowns)

# Retrieve risk-free rate data
rf = pdr.get_data_yahoo('^IRX', start_stocks, datetime.now())
rf = rf['Adj Close']

# Calculate excess returns of the portfolio
excess_returns = portf_allvalues.ptf_value_pctch.values / 100 - rf / 100 / 252
excess_returns = pd.Series(excess_returns).dropna()

# Calculate the average and standard deviation of excess returns
avg_excess_return = excess_returns.mean()
std_excess_return = excess_returns.std()

# Calculate the Sharpe ratio
sharpe_ratio = avg_excess_return / std_excess_return

# Get the 1-day return of the portfolio
return_1day = portf_allvalues.ptf_value_pctch.iloc[-1]

# Perform linear regression to calculate beta of the portfolio relative to the SP500
y = portf_allvalues.ptf_value_pctch.iloc[6:] / 100
x = portf_allvalues.sp500_pctch.iloc[6:] / 100
X = sm.add_constant(x)
model = sm.OLS(y, X)
results = model.fit()
beta = results.params[1].round(2)


[*********************100%%**********************]  1 of 1 completed
/var/folders/zq/cl9wyfm13sq150n30b6r3_6m0000gn/T/ipykernel_47779/1660114063.py:62: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [61]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from dash import dash_table
from jupyter_dash import JupyterDash

In [62]:
initial_date = '2020-01-03'  # do not use anything earlier than your first trade
plotlydf_portfval = portf_allvalues[portf_allvalues.index > initial_date]
plotlydf_portfval = plotlydf_portfval[['portf_value', 'sp500_mktvalue', 'ptf_value_pctch',
                                     'sp500_pctch', 'ptf_value_diff', 'sp500_diff']].reset_index().round(2)
# calculating cumulative growth since initial date
plotlydf_portfval['ptf_growth'] = plotlydf_portfval.portf_value/plotlydf_portfval['portf_value'].iloc[0]
plotlydf_portfval['sp500_growth'] = plotlydf_portfval.sp500_mktvalue/plotlydf_portfval['sp500_mktvalue'].iloc[0]
plotlydf_portfval.rename(columns={'index': 'date'}, inplace=True)  # needed for later
plotlydf_portfval

,date,portf_value,sp500_mktvalue,ptf_value_pctch,sp500_pctch,ptf_value_diff,sp500_diff,ptf_growth,sp500_growth
0,2022-01-04,51923.13,4793.54,NaN,NaN,NaN,NaN,1.000000,1.000000
1,2022-01-05,51020.65,4700.58,-1.74,-1.94,-902.48,-92.96,0.982619,0.980607
2,2022-01-06,50421.65,4696.05,-1.17,-0.10,-599.00,-4.53,0.971083,0.979662
3,2022-01-07,49847.82,4677.03,-1.14,-0.41,-573.83,-19.02,0.960031,0.975694
4,2022-01-10,49832.76,4670.29,-0.03,-0.14,-15.06,-6.74,0.959741,0.974288
...,...,...,...,...,...,...,...,...,...
596,2024-05-20,50336.00,5308.13,-0.86,0.09,-434.35,4.86,0.969433,1.107351
597,2024-05-21,50765.45,5321.41,0.85,0.25,429.45,13.28,0.977704,1.110121
598,2024-05-22,50438.80,5307.01,-0.64,-0.27,-326.65,-14.40,0.971413,1.107117
599,2024-05-23,49747.50,5267.84,-1.37,-0.74,-691.30,-39.17,0.958099,1.098946


In [63]:
CHART_THEME = 'plotly_white'  # others include seaborn, ggplot2, plotly_dark

chart_ptfvalue = go.Figure()  # generating a figure that will be updated in the following lines
chart_ptfvalue.add_trace(go.Scatter(x=plotlydf_portfval.date, y=plotlydf_portfval.portf_value,
                    mode='lines',  # you can also use "lines+markers", or just "markers"
                    name='Global Value'))
chart_ptfvalue.layout.template = CHART_THEME
chart_ptfvalue.layout.height=500
chart_ptfvalue.update_layout(margin = dict(t=50, b=50, l=25, r=25))  # this will help you optimize the chart space
chart_ptfvalue.update_layout(
    # title='Global Portfolio Value (USD $)',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Value: $ USD',
        titlefont_size=14,
        tickfont_size=12,
        ))
# chart_ptfvalue.update_xaxes(rangeslider_visible=False)
# chart_ptfvalue.update_layout(showlegend=False)
chart_ptfvalue.show()

In [64]:
import plotly.io as pio
list(pio.templates)  # doctest: +ELLIPSIS

['ggplot2',
 'seaborn',
 'simple_white',
 'plotly',
 'plotly_white',
 'plotly_dark',
 'presentation',
 'xgridoff',
 'ygridoff',
 'gridon',
 'none']

In [65]:
fig2 = go.Figure(data=[
    go.Bar(name='Portfolio', x=plotlydf_portfval['date'], y=plotlydf_portfval['ptf_value_pctch']),
    go.Bar(name='SP500', x=plotlydf_portfval['date'], y=plotlydf_portfval['sp500_pctch'])
])
# Change the bar mode
fig2.update_layout(barmode='group')
fig2.layout.template = CHART_THEME
fig2.layout.height=300
fig2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig2.update_layout(
#     title='% variation - Portfolio vs SP500',
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=14,
        tickfont_size=12,
        ))
fig2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))

fig2.show()

In [66]:
df = plotlydf_portfval[['date', 'ptf_growth', 'sp500_growth']].copy().round(3)
df['month'] = df.date.dt.month_name()  # date column should be formatted as datetime
df['weekday'] = df.date.dt.day_name()  # could be interesting to analyze weekday returns later
df['year'] = df.date.dt.year
df['weeknumber'] = df.date.dt.isocalendar().week    # could be interesting to try instead of timeperiod
df['timeperiod'] = df.year.astype(str) + ' - ' + df.date.dt.month.astype(str).str.zfill(2)
df.head(5)

,date,ptf_growth,sp500_growth,month,weekday,year,weeknumber,timeperiod
0,2022-01-04,1.000,1.000,January,Tuesday,2022,1,2022 - 01
1,2022-01-05,0.983,0.981,January,Wednesday,2022,1,2022 - 01
2,2022-01-06,0.971,0.980,January,Thursday,2022,1,2022 - 01
3,2022-01-07,0.960,0.976,January,Friday,2022,1,2022 - 01
4,2022-01-10,0.960,0.974,January,Monday,2022,2,2022 - 01


In [67]:
# getting the percentage change for each period. the first period will be NaN
sp = df.reset_index().groupby('timeperiod').last()['sp500_growth'].pct_change()*100
ptf = df.reset_index().groupby('timeperiod').last()['ptf_growth'].pct_change()*100
plotlydf_growth_compare = pd.merge(ptf, sp, on='timeperiod').reset_index().round(3)
plotlydf_growth_compare.head()

,timeperiod,ptf_growth,sp500_growth
0,2022 - 01,NaN,NaN
1,2022 - 02,-7.196,-3.185
2,2022 - 03,2.281,3.618
3,2022 - 04,-9.142,-8.783
4,2022 - 05,0.368,0.000


In [68]:
fig_growth2 = go.Figure()
fig_growth2.layout.template = CHART_THEME
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.ptf_growth.round(2),
    name='Portfolio'
))
fig_growth2.add_trace(go.Bar(
    x=plotlydf_growth_compare.timeperiod,
    y=plotlydf_growth_compare.sp500_growth.round(2),
    name='S&P 500',
))
fig_growth2.update_layout(barmode='group')
fig_growth2.layout.height=300
fig_growth2.update_layout(margin = dict(t=50, b=50, l=25, r=25))
fig_growth2.update_layout(
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='% change',
        titlefont_size=13,
        tickfont_size=12,
        ))

fig_growth2.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99))
fig_growth2.show()

In [69]:
returns_list = [kpi_portfolio7d_pct, kpi_portfolio15d_pct, kpi_portfolio30d_pct, kpi_portfolio200d_pct]

indicators_ptf = go.Figure()
indicators_ptf.layout.template = CHART_THEME

indicators_ptf.add_trace(go.Indicator(
    mode="number+delta",
    value= return_1day,# Set the initial value to the value for 1 day
    number={'suffix': " %", 'font': {'size': 30}},
    title={"text": "<br><span style='font-size:2em;color:gray'>Daily return</span>"},
    domain={'row': 0, 'column': 0}
))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = sharpe_ratio.round(2),
    number = {'font':{'size':30}},
    title = {"text": "<span style='font-size:2em;color:gray'>Sharpe ratio</span>"},
    domain = {'row': 1, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = beta,
    number = {'font':{'size':30}},
    title = {"text": "<span style='font-size:2em;color:gray'>Beta</span>"},
    domain = {'row': 2, 'column': 0}))

indicators_ptf.add_trace(go.Indicator(
    mode = "number+delta",
    value = te * 100,
    number = {'suffix': " %", 'font':{'size':30}},
    title = {"text": "<span style='font-size:2em;color:gray'>Tracking error</span>"},
    domain = {'row': 3, 'column': 1}))

indicators_ptf.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30),
)

In [70]:
indicators_ptf2 = go.Figure()
indicators_ptf2.layout.template = CHART_THEME
indicators_ptf2.add_trace(go.Indicator(
    mode = "number+delta",
    value = volatility,
    number = {'suffix': " %", 'font':{'size':30}},
    title = {"text": "<br><span style='font-size:2em;color:gray'>Volatility</span>"},
    domain = {'row': 0, 'column': 0}))

indicators_ptf2.add_trace(go.Indicator(
    mode = "number+delta",
    value = vaR_95,
    number = {'suffix': " %", 'font':{'size':30}},
    title = {"text": "<span style='font-size:2em;color:gray'>VaR 5d</span>"},
    domain = {'row': 1, 'column': 0}))

indicators_ptf2.add_trace(go.Indicator(
    mode = "number+delta",
    value = es_95,
    number = {'suffix': " %", 'font':{'size':30}},
    title = {"text": "<span style='font-size:2em;color:gray'>ES 5d</span>"},
    domain = {'row': 2, 'column': 0}))

indicators_ptf2.add_trace(go.Indicator(
    mode = "number+delta",
    value = max_drawdown * 100,
    number = {'suffix': " %", 'font':{'size':30}},
    title = {"text": "<span style='font-size:2em;color:gray'>Max drawdown</span>"},
    domain = {'row': 3, 'column': 1}))

indicators_ptf2.update_layout(
    grid = {'rows': 4, 'columns': 1, 'pattern': "independent"},
    margin=dict(l=50, r=50, t=30, b=30)
)

In [71]:
last_positions = final_filtered.groupby(['ticker']).agg({'cml_units': 'last', 'cml_cost': 'last',
                                                'gain_loss': 'sum', 'cashflow': 'sum'}).reset_index()

In [72]:
%%time
curr_prices = []
for tick in last_positions['ticker']:
    stonk = yf.Ticker(tick)
    price = stonk.info['regularMarketOpen']
    curr_prices.append(price)
    print(f'Done for {tick}')
len(curr_prices)

Done for AAPL
Done for AMZN
Done for GOOGL
Done for JNJ
Done for JPM
Done for MA
Done for PG
Done for PYPL
Done for TSLA
Done for UNH
Done for V
CPU times: user 40.2 ms, sys: 12.4 ms, total: 52.5 ms
Wall time: 747 ms


11

In [73]:
last_positions['price'] = curr_prices
last_positions['current_value'] = (last_positions.price * last_positions.cml_units).round(2)
last_positions['avg_price'] = (last_positions.cml_cost / last_positions.cml_units).round(2)
last_positions = last_positions.sort_values(by='current_value', ascending=False)

In [74]:
donut_top = go.Figure()
donut_top.layout.template = CHART_THEME
donut_top.add_trace(go.Pie(labels=last_positions.head(15).ticker, values=last_positions.head(15).current_value))
donut_top.update_traces(hole=0.7, hoverinfo="label+value+percent")
donut_top.update_traces(textposition='outside', textinfo='label+value')
donut_top.update_layout(showlegend=False)
donut_top.update_layout(margin = dict(t=50, b=50, l=25, r=25))
donut_top.show()

In [75]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('PORTFOLIO OVERVIEW', className='text-center text-primary, mb-3'))),  # header row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
            html.H5('Total Portfolio Value ($USD)', className='text-center'),
            dcc.Graph(id='chrt-portfolio-main',
                      figure=chart_ptfvalue,
                      style={'height':550}),
            html.Hr(),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on second row
            html.H5('Performance', className='text-center'),
            dcc.Graph(id='indicators-ptf',
                      figure=indicators_ptf,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 2}),  # width second column on second row
            dbc.Col([  # third column on second row
            html.H5('Risk metrics', className='text-center'),
            dcc.Graph(id='indicators-sp',
                      figure=indicators_ptf2,
                      style={'height':550}),
            html.Hr()
            ], width={'size': 2, 'offset': 0, 'order': 3}),  # width third column on second row
        ]),  # end of second row
        
        dbc.Row([  # start of third row
            dbc.Col([  # first column on third row
                html.H5('Monthly Return (%)', className='text-center'),
                dcc.Graph(id='chrt-portfolio-secondary',
                      figure=fig_growth2,
                      style={'height':380}),
            ], width={'size': 8, 'offset': 0, 'order': 1}),  # width first column on second row
            dbc.Col([  # second column on third row
                html.H5('Top 15 Holdings', className='text-center'),
                dcc.Graph(id='pie-top15',
                      figure = donut_top,
                      style={'height':380}),
            ], width={'size': 4, 'offset': 0, 'order': 2}),  # width second column on second row
        ])  # end of third row
        
    ], fluid=True)

if __name__ == "__main__":
    app.run_server(debug=True, port=8058)